### Incremental Data Ingestion

In [0]:
dbutils.widgets.text("src", "")

In [0]:
src_value = dbutils.widgets.get("src")

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS flightsproject.raw.bronze

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS flightsproject.raw.silver

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS flightsproject.raw.gold

In [0]:
df = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("cloudFiles.schemaLocation", f"/Volumes/flightsproject/bronze/bronzevolume/{src_value}/checkpoint") \
    .option("cloudFiles.schemaEvolutionMode", "rescue") \
    .load(f"/Volumes/flightsproject/raw/rawvolume/rawdata/{src_value}/")

In [0]:
df.writeStream.format("delta") \
    .outputMode("append") \
    .trigger(once=True) \
    .option("checkpointLocation", f"/Volumes/flightsproject/bronze/bronzevolume/{src_value}/checkpoint") \
    .option("path", f"/Volumes/flightsproject/bronze/bronzevolume/{src_value}/data") \
    .start()

In [0]:
%sql
SELECT * FROM delta.`/Volumes/flightsproject/bronze/bronzevolume/airports/data`